[Markdown Guide](https://docs.gitlab.com/ee/user/markdown.html)

# Praxismodul I
## Thema 6: Process Organisation
## Hochschule Mainz
## Wirtschaftsinformatik dual B.Sc., Fachsemester 4

Durchgeführt von

| Student | Matrikel-Nummer |
|:-- |:--|
|Moritz Wetzel | 943566|
|Till Waller | 943565|
Fabian Harmsen| 943611|



## Gliederung

1. Projektziel
2. Erklärungen zu den grafischen Ergebnissen
3. Configuration
4. Import
5. Similar Activities
6. Discover Roles
7. Handover Of Work
8. Clustering Similar Activities
9. Clustering Working together
10. Fazit

## 1. Projektziel

Im Rahmen dieser Projektarbeit wird die Prozessorganisation eines zur Verfügung gestellten Datensatzes untersucht. Die uns zur Verfügung gestellte Logdatei stammt aus der BPI-Challenge 2017 und bestehen aus 2 Geschäftsprozessen. Unser Projektziel wird daraus bestehen, soziale Netzwerke innerhalb eines gewünschten Datensatzes zu entdecken. Hierfür werden die Daten mit Methoden der sozialen Netzwerkanalyse untersucht, um die sozialen Verbindungen im importierten Datensatz zu untersuchen und auszuwerten. Die Ergebnisse werden mithilfe eines Jupyter-Notebooks berechnet und visuell dargestellt. Zum Ende des Projekts werden zwei weitere Datensätze mit den implementierten Analysen untersucht, um die Funktionalität der Methodik zu validieren.

## 2. Erklärung zu den grafischen Ergebnissen

Dieser Abschnitt erklärt die Grafischen Ergebnisse unabhängig von angewandten Methoden bzw. Analysen. Die Ausgaben setzen sich aus Nodes (Knoten) und Edges (Verbindungen) zusammen. Ein Node stellt im Rahmen unserer Arbeit einen Nutzer dar. Nach dem Clustering stellt ein Node eine Benutzergruppe dar. Die Benutzer beziehungsweise Benutzergruppen sind unterschiedlich stark über Edges miteinander verbunden. Die Stärke dieser Verbindung wird über Analysen festgestellt.

Aus Nodes und Edges werden Physiksimulationen erstellt, diese dienen der visuellen Darstellung der Methodenergebnisse. Die Edges wirken wie Gummibänder, die die Nodes zusammenhalten. Die Nodes hingegen versuchen möglichst viel Abstand zu anderen Nodes zu haben. Dies führt dazu, dass Nodes, die aufgrund der Analysen mit starken Edges verbunden sind, sich annähern und weniger stark verbundene Nodes sich voneinander entfernen. Sobald sich die Physiksimulation beruhigt hat, kann man Gruppen von Nodes erkennen sowie die Verbindungen zu anderen Gruppen.

Wir bieten zwei Methoden zur erstellung von Grafischen Ausgaben, "NetworkX" wird direkt im Notebook ausgegeben, "Pyvis" erzeugt genauere webbasierte Ausgaben.
Diese Analyseergebnisse können als HTML-Datei im Browser geöffnet werden und sind unter "result/*" zu finden.

*Wir erzeugen beim beziehungsweise nach dem Clustering keine grafische Ausgabe, da dies bei den getesteten Datensätzen keine neuen Erkenntnisse gebracht hat.*

## 3. Configuration

Die folgende Tabelle erklärt die Variablen, die im nächsten Code-Block definiert werden können. Die Logdatei muss, um richtig analysiert werden zu können, im XES-Dateiformat vorliegen. Der Speicherort muss in der Variable `log_path` angegeben werden.

| variable | type | description| default|
|:--|:--|:--|:--|
|log_path | (path)| path to the XES File|`data/test.xes`|
|use_networkx | (bool)| output via NetworkX or Pyvis| `False`|
|filter_timeframe | (bool)| filter the timeframe of the traces| `False` |
|filter_timeframe_intersecting | (bool)| use intersecting or contained| `False` |
|filter_timeframe_start | (dateTime)| start of the Timeframe| `1970-01-01 00:00:00` |
|filter_timeframe_end | (dateTime)| end of the Timeframe| `2038-12-31 00:00:00` |
|filter_performance| (bool)| filter on Traceperformance| `False` |
|filter_performance_min| (int)| minimum seconds of trace actions|`0` |
|filter_performance_max |(int)| maximum seconds of trace actions|`360000` |

Die Option `Pyvis` legt die Ergebnisse im Ordner `./result` ab, welcher schreibend Zugriff benötigt.

In [ ]:
log_path = 'data/BPI_Challenge_2017_Offer_log.xes'

use_networkx = False

filter_timeframe = False
filter_timeframe_intersecting  = False
filter_timeframe_start = "2011-03-09 00:00:00"
filter_timeframe_end = "2011-03-09 00:00:00"

filter_performance = False
filter_performance_min = 0
filter_performance_max = 864000

## 4. Import
Im folgenden Codeblock werden alle Bibliotheken geladen die im laufenden Skript benötigt werden. Wenn die Bibliotheken nicht gefunden werden können, können sie über den Packagemanager pip seperat nachinstalliert werden.

**Unvollständige Liste der Pythonpackete (pip)**
 - pm4py
 - shutil
 - numpy
 - sklearn
 - scipy


In [ ]:
import shutil
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.organizational_mining.sna import algorithm as sna
from pm4py.visualization.sna import visualizer as sna_visualizer
from pm4py.algo.organizational_mining.roles import algorithm as roles_discovery
from pm4py.algo.organizational_mining.sna import util
from pm4py.algo.filtering.log.timestamp import timestamp_filter
from pm4py.algo.filtering.log.cases import case_filter
from pm4py.algo.organizational_mining.local_diagnostics import algorithm as local_diagnostics
from pathlib import Path

Der Ordner für die webbasierten Exportergebnisse wird in diesem Codeblock erstellt. Wird NetworkX als Ausgabe genutzt, wird dieser Abschnitt übersprungen.


In [ ]:
if not use_networkx:
    Path("./result").mkdir(parents=True, exist_ok=True)

Dieser Code-Block lädt den Datensatz und wendet die eingestellten Filter an.
Die Einstellungen dazu wurden in Block 3 (Configuration) festgelegt.

In [ ]:
log = xes_importer.apply(log_path)

if filter_timeframe:
    if not filter_timeframe_intersecting:
        log = timestamp_filter.filter_traces_contained(   log, filter_timeframe_start, filter_timeframe_end)
    else:
        log = timestamp_filter.filter_traces_intersecting(log, filter_timeframe_start, filter_timeframe_end)

if filter_performance:
    log = case_filter.filter_case_performance(log, filter_performance_min, filter_performance_max)

## 5. Similar Activities

In diesem Abschnitt wird die Methode "Similar Activities" auf den zuvor importierten Datensatz angewendet, um zu berechnen, wie sehr sich die Arbeitsabläufe zwischen den Nutzern (Nodes) ähneln (Quelle: https://pm4py.fit.fraunhofer.de/documentation).

In [ ]:
sa_metric  = sna.apply(log, variant=sna.Variants.JOINTACTIVITIES_LOG)

if use_networkx:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_ja_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_ja_py, "result/similar_activities.html")

## 6. Discover Roles

In diesem Abschnitt wird die Methode “Discover Roles” auf den zuvor importierten Datensatz angewendet, um die Rollen eines Nutzers (Node) anhand seiner durchgeführten Aktivitäten zu erkennen. Eine Rolle ist dabei ein Satz von Arbeitsaktivitäten im Log, die in ähnlicher Weise von einem Nutzer (Node) oder mehreren Nutzern (Nodes) ausgeführt wird. Zu Beginn der Analyse gehört jede Arbeitsaktivität zu unterschiedlichen Positionen und ist mit einer hohen Anzahl an Urhebern verbunden. Anschließend werden die Rollen entsprechend ihrer Ähnlichkeit der Arbeitsabläufe gruppiert, bis keine weitere Gruppierung mehr möglich ist. (Quelle: https://pm4py.fit.fraunhofer.de/documentation)

Folgende Quelle kann helfen, die aufgeführten Rollen zu verstehen und zu erkennen:

Burattin, Andrea, Alessandro Sperduti, and Marco Veluscek. “Business models enhancement through discovery of roles.” 2013 IEEE Symposium on Computational Intelligence and Data Mining (CIDM). IEEE, 2013

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Rollenname auf Basis der Aktivitäten
2. Benutzer mit der Angabe wie aktiv der Nutzer in der Rolle war
3. `Leerzeile`

In [ ]:
roles = roles_discovery.apply(log)

for x in roles:
    print(x[0])
    print(x[1])
    print("")

## 7. Handover Of Work

In diesem Abschnitt wird die Methode “Handover Of Work” auf den zuvor importierten Datensatz angewendet, um zu messen, wie oft ein Nutzer (Node) bei der Ausführung eines Geschäftsprozesses mit einer anderen Person zusammenarbeitet. (Quelle: https://pm4py.fit.fraunhofer.de/documentation)
Ein HandOver wird hier als folgendes definiert, ein Nutzer übergibt den Geschäftsvorfall am Ende seiner Aktivität an einen anderen Nutzer. Die beiden Nutzer sind dabei die Nodes, die Edges sind eine Metrik für die Anzahl an übergebenden Geschäftsvorfällen. Die Edges zeigen bei “Handover Of Work” Analysen somit auch die Richtung des Arbeitsflusses.



In [ ]:
hw_values = sna.apply(log, variant=sna.Variants.HANDOVER_LOG)

if use_networkx:
    gviz_hw_py = sna_visualizer.apply(hw_values, variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_hw_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_hw_py = sna_visualizer.apply(hw_values , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_hw_py, "result/handover_of_work.html")

## 8. Clustering Similar Activities

In diesem Abschnitt werden die unter 6 analysierten Nutzer in Nutzergruppen zusammengefasst, welche gleiche Aufgaben verrichtet haben. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [ ]:
clustering_sa = util.cluster_affinity_propagation(sa_metric)

for group in clustering_sa:
    print("Gruppe: " + group)
    print(clustering_sa[group])
    print("")


## 9. Clustering Handover Of Work

In diesem Abschnitt werden die unter 8 analysierten Nutzer als Gruppe zusammengefasst, welche häufig miteinander arbeiten. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [ ]:
clustering_hw = util.cluster_affinity_propagation(hw_values)
for group in clustering_hw:
    print("Gruppe: " + group)
    print(clustering_hw[group])
    print("")

## 10. Fazit

TEXT TEXT TEXT

In [ ]:
clustering_sa = util.cluster_affinity_propagation(sa_metric)

for group in clustering_sa:
    print("Gruppe: " + group)
    print(clustering_sa[group])
    print("")


## 9. Clustering Handover Of Work

In diesem Abschnitt werden die unter 8 analysierten Nutzer als Gruppe zusammengefasst, welche häufig miteinander arbeiten. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [ ]:
clustering_hw = util.cluster_affinity_propagation(hw_values)
for group in clustering_hw:
    print("Gruppe: " + group)
    print(clustering_hw[group])
    print("")

## 10. Fazit

Im Projekt konten wir aus verschiedensten Datensätze auswerten. Dabei konnte man sowohl Guppenpersonen erkennen die regelmäßig die gleichen Aktivitäten machen aber auch Personen die an verschiedenen Stellen in einem Geschäftsprozess arbeiten.
Auffällig war aber, dass es auch Geschäftsprozesse gab bei denen alle Beteiligten ein großes Team waren, in solchen Fällen erzeugen die durchgeführten Analysen kein interpretierbares Ergebnis.


## 10. Fazit

TEXT TEXT TEXT

In [8]:
clustering_sa = util.cluster_affinity_propagation(sa_metric)

for group in clustering_sa:
    print("Gruppe: " + group)
    print(clustering_sa[group])
    print("")


Gruppe: -1
['User_1', 'User_10', 'User_100', 'User_101', 'User_102', 'User_103', 'User_104', 'User_105', 'User_106', 'User_107', 'User_108', 'User_109', 'User_11', 'User_110', 'User_111', 'User_112', 'User_113', 'User_114', 'User_115', 'User_116', 'User_117', 'User_118', 'User_119', 'User_12', 'User_120', 'User_121', 'User_122', 'User_123', 'User_124', 'User_125', 'User_126', 'User_127', 'User_128', 'User_129', 'User_13', 'User_130', 'User_131', 'User_132', 'User_133', 'User_134', 'User_135', 'User_136', 'User_137', 'User_138', 'User_139', 'User_14', 'User_140', 'User_141', 'User_142', 'User_143', 'User_144', 'User_15', 'User_16', 'User_17', 'User_18', 'User_19', 'User_2', 'User_20', 'User_21', 'User_22', 'User_23', 'User_24', 'User_25', 'User_26', 'User_27', 'User_28', 'User_29', 'User_3', 'User_30', 'User_31', 'User_32', 'User_33', 'User_34', 'User_35', 'User_36', 'User_37', 'User_38', 'User_39', 'User_4', 'User_40', 'User_41', 'User_42', 'User_43', 'User_44', 'User_45', 'User_46', '

/Users/moritzwetzel/PycharmProjects/PraxisModul1_PM4PY/venv/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(
/Users/moritzwetzel/PycharmProjects/PraxisModul1_PM4PY/venv/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:528: ConvergenceWarning: This model does not have any cluster centers because affinity propagation did not converge. Labeling every sample as '-1'.
  warnings.warn(


## 9. Clustering Handover Of Work

In diesem Abschnitt werden die unter 8 analysierten Nutzer als Gruppe zusammengefasst, welche häufig miteinander arbeiten. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [9]:
clustering_hw = util.cluster_affinity_propagation(hw_values)
for group in clustering_hw:
    print("Gruppe: " + group)
    print(clustering_hw[group])
    print("")

Gruppe: 4
['User_1', 'User_100', 'User_101', 'User_102', 'User_103', 'User_104', 'User_105', 'User_106', 'User_107', 'User_108', 'User_109', 'User_110', 'User_111', 'User_114', 'User_115', 'User_124', 'User_125', 'User_127', 'User_128', 'User_129', 'User_13', 'User_130', 'User_131', 'User_132', 'User_133', 'User_134', 'User_135', 'User_136', 'User_137', 'User_138', 'User_139', 'User_140', 'User_141', 'User_142', 'User_143', 'User_144', 'User_20', 'User_22', 'User_23', 'User_25', 'User_26', 'User_29', 'User_30', 'User_31', 'User_33', 'User_34', 'User_43', 'User_45', 'User_47', 'User_50', 'User_54', 'User_55', 'User_58', 'User_59', 'User_6', 'User_62', 'User_63', 'User_64', 'User_65', 'User_66', 'User_68', 'User_69', 'User_70', 'User_72', 'User_73', 'User_74', 'User_75', 'User_76', 'User_77', 'User_79', 'User_8', 'User_80', 'User_81', 'User_82', 'User_83', 'User_84', 'User_85', 'User_86', 'User_87', 'User_88', 'User_89', 'User_9', 'User_90', 'User_91', 'User_92', 'User_93', 'User_94', 'U

## 10. Fazit

TEXT TEXT TEXT